## Installing tensorflow

In [2]:
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable


## Installing scikit-learn

In [1]:
!pip install scikit-learn-intelex
from sklearnex import patch_sklearn
patch_sklearn()

Defaulting to user installation because normal site-packages is not writeable


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score,confusion_matrix

## Importing tensorflow

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

2023-08-10 04:54:42.596689: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-10 04:54:42.598368: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-10 04:54:42.628664: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-10 04:54:42.629325: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-10 04:54:45.463477: W tensorflow/compiler/tf2t

## Data Preparation

In [4]:
# Set up the data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_labels = pd.read_csv('normal_train/train_annotations.csv')
# Convert the column to strings
train_labels['class'] = train_labels['class'].astype(str)
train_generator = train_datagen.flow_from_dataframe(
    train_labels,
    directory='normal_train',
    x_col='filename',
    y_col='class',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

Found 9177 validated image filenames belonging to 5 classes.


/home/u117015/.local/lib/python3.10/site-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 3639 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


## Installing Pillow

In [5]:
pip install Pillow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
import PIL
print(PIL.__version__)

10.0.0


In [7]:
from tensorflow.keras.preprocessing.image import load_img


## Model Creation

In [8]:
base_model = tf.keras.applications.DenseNet169(
    include_top=False, weights='imagenet', input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

# Add new fully connected layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Compile the model with binary cross-entropy loss and Adam optimizer
model = Model(inputs=base_model.input, outputs=predictions)

# freeze Layers
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy',
              metrics=['accuracy'])

# training the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20)


Epoch 1/20
286/286 [==============================] - 98s 327ms/step - loss: 0.5105 - accuracy: 0.7981
Epoch 2/20
286/286 [==============================] - 90s 314ms/step - loss: 0.5024 - accuracy: 0.8000
Epoch 3/20
286/286 [==============================] - 90s 313ms/step - loss: 0.5018 - accuracy: 0.8000
Epoch 4/20
286/286 [==============================] - 93s 325ms/step - loss: 0.5015 - accuracy: 0.8000
Epoch 5/20
286/286 [==============================] - 96s 335ms/step - loss: 0.5014 - accuracy: 0.8000
Epoch 6/20
286/286 [==============================] - 96s 334ms/step - loss: 0.5011 - accuracy: 0.8000
Epoch 7/20
286/286 [==============================] - 95s 333ms/step - loss: 0.5012 - accuracy: 0.8000
Epoch 8/20
286/286 [==============================] - 95s 332ms/step - loss: 0.5012 - accuracy: 0.8000
Epoch 9/20
286/286 [==============================] - 95s 332ms/step - loss: 0.5010 - accuracy: 0.8000
Epoch 10/20
286/286 [==============================] - 95s 332ms/step - l

In [9]:
t_test_df = pd.read_csv("normal_test/_annotations.csv")

In [10]:
# Create a generator for the test images
t_test_datagen = ImageDataGenerator(rescale=1./255)
t_test_generator = t_test_datagen.flow_from_dataframe(
    t_test_df,
    directory="normal_test",
    x_col='filename',
    y_col=None,
    target_size=(224, 224),
    batch_size=32,
    class_mode=None,
    shuffle=False)

# Use the trained model to make predictions on the test data
preds = model.predict_generator(t_test_generator, steps=None)

# Remove any missing filenames from the test CSV file
missing_filenames = set(t_test_generator.filenames) - set(t_test_df['filename'])
if missing_filenames:
    print(f'Removing {len(missing_filenames)} missing filenames from the test CSV file.')
    t_test_df = t_test_df[~t_test_df['filename'].isin(missing_filenames)]

Found 823 validated image filenames.


/tmp/ipykernel_667181/3169443909.py:14: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  preds = model.predict_generator(t_test_generator, steps=None)


In [11]:
li = []
for i in preds.ravel():
    if i>0.5:
        li.append(1)
    else:
        li.append(0)
     
# Create a DataFrame with the filenames and predictions
df = pd.DataFrame({'filename': t_test_generator.filenames[:len(preds)], 'class': li})

test_inst_df=pd.merge(df, t_test_df, on='filename', how='inner')

## Saving the trained Model

In [12]:
# Save the model
model.save('final_models_densenet/densenet.h5')

/home/u117015/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
